In [1]:
# General imports
import numpy as np
import pandas as pd
import os, sys, gc, time, warnings, pickle, psutil, random

warnings.filterwarnings('ignore')

In [2]:
# :seed to make all processes deterministic     # type: int
def seed_everything(seed=0):
    random.seed(seed)
    np.random.seed(seed)

In [3]:
# Read data
def get_data_by_store(store):
    
    # Read and contact basic feature
    df = pd.concat([pd.read_pickle(BASE),
                    pd.read_pickle(PRICE).iloc[:,2:],
                    pd.read_pickle(CALENDAR).iloc[:,2:]],
                    axis=1)
    
    # Leave only relevant store
    df = df[df['store_id']==store]

    # With memory limits we have to read 
    # lags and mean encoding features
    # separately and drop items that we don't need.
    # As our Features Grids are aligned 
    # we can use index to keep only necessary rows
    # Alignment is good for us as concat uses less memory than merge.
    df2 = pd.read_pickle(MEAN_ENC)[mean_features]
    df2 = df2[df2.index.isin(df.index)]
    
    df3 = pd.read_pickle(LAGS).iloc[:,3:]
    df3 = df3[df3.index.isin(df.index)]
    
    df = pd.concat([df, df2], axis=1)
    del df2 # to not reach memory limit 
    
    df = pd.concat([df, df3], axis=1)
    del df3 # to not reach memory limit 
    
    # Create features list
    features = [col for col in list(df) if col not in remove_features]
    df = df[['id','d',TARGET]+features]
    
    # Skipping first n rows
    df = df[df['d']>=START_TRAIN].reset_index(drop=True)
    
    return df, features

# Recombine Test set after training
def get_base_test():
    base_test = pd.DataFrame()

    for store_id in STORES_IDS:
        temp_df = pd.read_pickle('test_'+store_id+str(VER)+'.pkl')
        temp_df['store_id'] = store_id
        base_test = pd.concat([base_test, temp_df]).reset_index(drop=True)
    
    return base_test


########################### Helper to make dynamic rolling lags
#################################################################################
def make_lag(LAG_DAY):
    lag_df = base_test[['id','d',TARGET]]
    col_name = 'sales_lag_'+str(LAG_DAY)
    lag_df[col_name] = lag_df.groupby(['id'])[TARGET].transform(lambda x: x.shift(LAG_DAY)).astype(np.float16)
    return lag_df[[col_name]]

In [4]:
def make_lag_roll(LAG_DAY,lag_df_new):
   
    lag_df = base_test[['id','d',TARGET]]
    for i in range(0,len(LAG_DAY)):
        
        shift_day = LAG_DAY[i][0]
        roll_wind = LAG_DAY[i][1]
        col_name = 'rolling_mean_tmp_'+str(shift_day)+'_'+str(roll_wind)
        lag_df[col_name] = lag_df.groupby(['id'])[TARGET].transform(lambda x: x.shift(shift_day).rolling(roll_wind).mean())
    
    lag_df_new=lag_df[lag_df.columns[3:]].copy()
    return lag_df_new

In [5]:
import xgboost as xgb
xgb_params = {
                    'booster': 'gbtree',
                    'objective': 'reg:tweedie',
                    'tweedie_variance_power': 1.1,
                    'metric': 'rmse',
                    'subsample': 0.5,
                    'eta': 0.03,
                    "lambda":0.1,
                    'max_leaves': 2**11-1,
                    'min_child_weight': 2,
                    'colsample_bytree': 0.5,
                    'max_bin': 100,
                    'n_estimators': 1400,
                } 



# lgb_params ={
#         "objective" : "tweedie",
#         "metric" :"rmse",
#         "force_row_wise" : True,
#         "learning_rate" : 0.075,
#         "sub_feature" : 0.8,
#         "sub_row" : 0.75,
#         "bagging_freq" : 1,
#         "lambda_l2" : 0.1,
#         "metric": ["rmse"],
#         "nthread": -1,
#         "tweedie_variance_power":1.1,
#     'verbosity': 1,
# #     'num_iterations' : 1500,
#     'num_leaves': 128,
#     "min_data_in_leaf": 104,
#     }




# Let's look closer on params

## 'boosting_type': 'gbdt'
# we have 'goss' option for faster training
# but it normally leads to underfit.
# Also there is good 'dart' mode
# but it takes forever to train
# and model performance depends 
# a lot on random factor 
# https://www.kaggle.com/c/home-credit-default-risk/discussion/60921

## 'objective': 'tweedie'
# Tweedie Gradient Boosting for Extremely
# Unbalanced Zero-inflated Data
# https://arxiv.org/pdf/1811.10192.pdf
# and many more articles about tweediie
#
# Strange (for me) but Tweedie is close in results
# to my own ugly loss.
# My advice here - make OWN LOSS function
# https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/140564
# https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/143070
# I think many of you already using it (after poisson kernel appeared) 
# (kagglers are very good with "params" testing and tuning).
# Try to figure out why Tweedie works.
# probably it will show you new features options
# or data transformation (Target transformation?).

## 'tweedie_variance_power': 1.1
# default = 1.5
# set this closer to 2 to shift towards a Gamma distribution
# set this closer to 1 to shift towards a Poisson distribution
# my CV shows 1.1 is optimal 
# but you can make your own choice

## 'metric': 'rmse'
# Doesn't mean anything to us
# as competition metric is different
# and we don't use early stoppings here.
# So rmse serves just for general 
# model performance overview.
# Also we use "fake" validation set
# (as it makes part of the training set)
# so even general rmse score doesn't mean anything))
# https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/133834

## 'subsample': 0.5
# Serves to fight with overfit
# this will randomly select part of data without resampling
# Chosen by CV (my CV can be wrong!)
# Next kernel will be about CV

##'subsample_freq': 1
# frequency for bagging
# default value - seems ok

## 'learning_rate': 0.03
# Chosen by CV
# Smaller - longer training
# but there is an option to stop 
# in "local minimum"
# Bigger - faster training
# but there is a chance to
# not find "global minimum" minimum

## 'num_leaves': 2**11-1
## 'min_data_in_leaf': 2**12-1
# Force model to use more features
# We need it to reduce "recursive"
# error impact.
# Also it leads to overfit
# that's why we use small 

# 'max_bin': 100
## l1, l2 regularizations
# https://towardsdatascience.com/l1-and-l2-regularization-methods-ce25e7fc831c
# Good tiny explanation
# l2 can work with bigger num_leaves
# but my CV doesn't show boost
                    
## 'n_estimators': 1400
# CV shows that there should be
# different values for each state/store.
# Current value was chosen 
# for general purpose.
# As we don't use any early stopings
# careful to not overfit Public LB.

##'feature_fraction': 0.5
# LightGBM will randomly select 
# part of features on each iteration (tree).
# We have maaaany features
# and many of them are "duplicates"
# and many just "noise"
# good values here - 0.5-0.7 (by CV)

## 'boost_from_average': False
# There is some "problem"
# to code boost_from_average for 
# custom loss
# 'True' makes training faster
# BUT carefull use it
# https://github.com/microsoft/LightGBM/issues/1514

In [6]:
VER = 5                         # Our model version
SEED = 666                      # We want all things
seed_everything(SEED)            # to be as deterministic       # as possible
N_CORES = psutil.cpu_count()     # Available CPU cores


#LIMITS and const
TARGET      = 'sales'            # Our target
START_TRAIN = 0                  # We can skip some rows (Nans/faster training)
END_TRAIN   = 1913               # End day of our train set
P_HORIZON   = 28                 # Prediction horizon

#FEATURES to remove
## These features lead to overfit
## or values not present in test set
remove_features = ['id','state_id','store_id',
                   'date','wm_yr_wk','d',TARGET,'cluster']
mean_features   = ['enc_cat_id_mean','enc_cat_id_std',
                   'enc_dept_id_mean','enc_dept_id_std',
                   'enc_item_id_mean','enc_item_id_std'] 

#PATHS for Features
BASE     = 'grid_part_1.pkl'
PRICE    = 'grid_part_2.pkl'
CALENDAR = 'grid_part_3.pkl'
LAGS     = 'lags_df_28_v2.pkl'
MEAN_ENC = 'mean_encoding_df.pkl'


# AUX(pretrained) Models paths

#STORES ids
STORES_IDS = pd.read_csv('sales_train_validation.csv')['store_id']
STORES_IDS = list(STORES_IDS.unique())

#SPLITS for lags creation
SHIFT_DAY  = 28
N_LAGS     = 15
LAGS_SPLIT = [col for col in range(SHIFT_DAY,SHIFT_DAY+N_LAGS)]
ROLS_SPLIT = []
for i in [1,7,14]:
    for j in [7,14,28,56]:
        ROLS_SPLIT.append([i,j])

In [7]:
for store_id in STORES_IDS:
    print('Train', store_id)
    
    # Get grid for current store
    grid_df, features_columns = get_data_by_store(store_id)
    
    print(features_columns)
    cat_cols1=['item_id', 'dept_id','cat_id', 'event_name_1','event_type_1', 'event_name_2', 'event_type_2']
    for i, v in tqdm(enumerate(cat_cols1)):
        grid_df[v]=grid_df[v].cat.add_categories('Missing')
        grid_df[v].fillna('Missing', inplace=True)
        grid_df[v] = OrdinalEncoder(dtype="int").fit_transform(grid_df[[v]])
    cat_cols2=['NBA','event_after_weekend','event_bef_weekend','event_attention_after','event_attention_bef',
               'snap_CA','snap_TX','snap_WI','price_rank_dept','is_first_half_month']
    for i, v in tqdm(enumerate(cat_cols2)):
        grid_df[v]=pd.to_numeric(grid_df[v])
    # Masks for 
    # Train (All data less than 1913)
    # "Validation" (Last 28 days - not real validation set)
    # Test (All data greater than 1913 day, 
    #       with some gap for recursive features)
    train_mask = grid_df['d']<=END_TRAIN
    valid_mask = train_mask&(grid_df['d']>(END_TRAIN-P_HORIZON))
    preds_mask = grid_df['d']>(END_TRAIN-100)
    
    # Apply masks and save lgb dataset as bin
    # to reduce memory spikes during dtype convertations
    # https://github.com/Microsoft/LightGBM/issues/1032
    # "To avoid any conversions, you should always use np.float32"
    # or save to bin before start training
    # https://www.kaggle.com/c/talkingdata-adtracking-fraud-detection/discussion/53773
    train_data = xgb.DMatrix(grid_df[train_mask][features_columns], 
                       label=grid_df[train_mask][TARGET])
    
    
    valid_data = xgb.DMatrix(grid_df[valid_mask][features_columns], 
                       label=grid_df[valid_mask][TARGET])
    
    watchlist = [(train_data, 'train'), (valid_data, 'valid')]
    
    # Saving part of the dataset for later predictions
    # Removing features that we need to calculate recursively 
    grid_df = grid_df[preds_mask].reset_index(drop=True)
    keep_cols = [col for col in list(grid_df) if '_tmp_' not in col]
    grid_df = grid_df[keep_cols]
    grid_df.to_pickle('test_'+store_id+str(VER)+'.pkl')
    del grid_df
    gc.collect()
    
    # Launch seeder again to make lgb training 100% deterministic
    # with each "code line" np.random "evolves" 
    # so we need (may want) to "reset" it
    seed_everything(SEED)
    estimator = xgb.train(xgb_params,
                          train_data,
                          watchlist,
                          verbose_eval = 100,
                          )
    
    # Save model - it's not real '.bin' but a pickle file
    # estimator = lgb.Booster(model_file='model.txt')
    # can only predict with the best iteration (or the saving iteration)
    # pickle.dump gives us more flexibility
    # like estimator.predict(TEST, num_iteration=100)
    # num_iteration - number of iteration want to predict with, 
    # NULL or <= 0 means use best iteration
    model_name = 'xgb_model_'+store_id+'_v'+str(VER)+'.bin'
    pickle.dump(estimator, open(model_name, 'wb'))

    # Remove temporary files and objects 
    # to free some hdd space and ram memory
    #     !rm train_data.bin
    del train_data, valid_data, estimator
    gc.collect()
    
    # "Keep" models features for predictions
    MODEL_FEATURES = features_columns

Train CA_1
['item_id', 'dept_id', 'cat_id', 'release', 'sell_price', 'price_max', 'price_min', 'price_std', 'price_mean', 'price_norm', 'price_rank_dept', 'price_nunique', 'item_nunique', 'price_momentum', 'price_momentum_m', 'price_momentum_y', 'temperature_high', 'temperature_con', 'rainfall_m', 'snow_m', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI', 'is_first_half_month', 'event_bef_weekend', 'event_after_weekend', 'NBA', 'event_attention_after', 'event_attention_bef', 'event_attention_sum', 'tm_d', 'tm_w', 'tm_m', 'tm_q', 'tm_y', 'tm_wm', 'tm_dw', 'tm_w_end', 'enc_cat_id_mean', 'enc_cat_id_std', 'enc_dept_id_mean', 'enc_dept_id_std', 'enc_item_id_mean', 'enc_item_id_std', 'sales_lag_28', 'sales_lag_29', 'sales_lag_30', 'sales_lag_31', 'sales_lag_32', 'sales_lag_33', 'sales_lag_34', 'sales_lag_35', 'sales_lag_36', 'sales_lag_37', 'sales_lag_38', 'sales_lag_39', 'sales_lag_40', 'sales_lag_41', 'sales_lag_42', 'rolling_mean_7', 'rolli

[100]	valid_0's rmse: 1.6106
[200]	valid_0's rmse: 1.59341
[300]	valid_0's rmse: 1.58578
[400]	valid_0's rmse: 1.57942
[500]	valid_0's rmse: 1.57492
[600]	valid_0's rmse: 1.57055
[700]	valid_0's rmse: 1.56631
[800]	valid_0's rmse: 1.5622
[900]	valid_0's rmse: 1.55854
[1000]	valid_0's rmse: 1.55492
[1100]	valid_0's rmse: 1.5519
[1200]	valid_0's rmse: 1.54882
[1300]	valid_0's rmse: 1.54545
[1400]	valid_0's rmse: 1.54238
Train TX_2
['item_id', 'dept_id', 'cat_id', 'release', 'sell_price', 'price_max', 'price_min', 'price_std', 'price_mean', 'price_norm', 'price_rank_dept', 'price_nunique', 'item_nunique', 'price_momentum', 'price_momentum_m', 'price_momentum_y', 'temperature_high', 'temperature_con', 'rainfall_m', 'snow_m', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI', 'is_first_half_month', 'event_bef_weekend', 'event_after_weekend', 'NBA', 'event_attention_after', 'event_attention_bef', 'event_attention_sum', 'tm_d', 'tm_w', 'tm_m', 'tm

[100]	valid_0's rmse: 1.92611
[200]	valid_0's rmse: 1.8521
[300]	valid_0's rmse: 1.83399
[400]	valid_0's rmse: 1.8214
[500]	valid_0's rmse: 1.81218
[600]	valid_0's rmse: 1.80483
[700]	valid_0's rmse: 1.79628
[800]	valid_0's rmse: 1.79
[900]	valid_0's rmse: 1.78537
[1000]	valid_0's rmse: 1.77923
[1100]	valid_0's rmse: 1.77444
[1200]	valid_0's rmse: 1.76995
[1300]	valid_0's rmse: 1.76523
[1400]	valid_0's rmse: 1.762


In [8]:
# Create Dummy DataFrame to store predictions
all_preds = pd.DataFrame()

# Join back the Test dataset with 
# a small part of the training data 
# to make recursive features
base_test = get_base_test()

# Timer to measure predictions time 
main_time = time.time()

# Loop over each prediction day
# As rolling lags are the most timeconsuming
# we will calculate it for whole day


for PREDICT_DAY in range(1,29):    
    print('Predict | Day:', PREDICT_DAY)
    start_time = time.time()

    # Make temporary grid to calculate rolling lags
    grid_df = base_test.copy()
    
    
    lag_df_new = pd.DataFrame()

    lag_df_new=make_lag_roll(ROLS_SPLIT,lag_df_new)


    grid_df = pd.concat([grid_df, lag_df_new], axis=1)


    for store_id in STORES_IDS:

        # Read all our models and make predictions
        # for each day/store pairs
        model_path = 'xgb_model_'+store_id+'_v'+str(VER)+'.bin' 

        estimator = pickle.load(open(model_path, 'rb'))

        day_mask = base_test['d']==(END_TRAIN+PREDICT_DAY)
        store_mask = base_test['store_id']==store_id

        mask = (day_mask)&(store_mask)
        base_test[TARGET][mask] = estimator.predict(xgb.DMatrix(grid_df[mask][MODEL_FEATURES]))

    # Make good column naming and add 
    # to all_preds DataFrame
    temp_df = base_test[day_mask][['id',TARGET]]
    temp_df.columns = ['id','F'+str(PREDICT_DAY)]
    if 'id' in list(all_preds):
        all_preds = all_preds.merge(temp_df, on=['id'], how='left')
    else:
        all_preds = temp_df.copy()

    print('#'*10, ' %0.2f min round |' % ((time.time() - start_time) / 60),
                  ' %0.2f min total |' % ((time.time() - main_time) / 60),
                  ' %0.2f day sales |' % (temp_df['F'+str(PREDICT_DAY)].sum()))
    
    del temp_df, lag_df_new

all_preds = all_preds.reset_index(drop=True)
all_preds.head()

Predict | Day: 1
##########  4.10 min round |  4.10 min total |  37507.17 day sales |
Predict | Day: 2
##########  4.21 min round |  8.31 min total |  35512.83 day sales |
Predict | Day: 3
##########  4.20 min round |  12.51 min total |  35256.69 day sales |
Predict | Day: 4
##########  4.05 min round |  16.56 min total |  35651.27 day sales |
Predict | Day: 5
##########  3.52 min round |  20.08 min total |  42123.12 day sales |
Predict | Day: 6
##########  3.50 min round |  23.58 min total |  50993.88 day sales |
Predict | Day: 7
##########  3.53 min round |  27.11 min total |  53927.77 day sales |
Predict | Day: 8
##########  3.58 min round |  30.69 min total |  43986.83 day sales |
Predict | Day: 9
##########  3.54 min round |  34.23 min total |  43828.20 day sales |
Predict | Day: 10
##########  3.59 min round |  37.82 min total |  38606.66 day sales |
Predict | Day: 11
##########  3.55 min round |  41.37 min total |  41075.00 day sales |
Predict | Day: 12
##########  3.58 min roun

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0.851964,0.742018,0.759910,0.773099,0.992357,1.102528,1.180837,0.839067,0.881795,...,0.815830,1.108235,1.061415,0.792533,0.707089,0.719586,0.743990,0.872711,1.041275,0.935274
1,HOBBIES_1_002_CA_1_validation,0.207244,0.184524,0.163541,0.184283,0.201804,0.279155,0.334076,0.211482,0.193800,...,0.212989,0.256653,0.245208,0.190491,0.166705,0.187177,0.180707,0.190131,0.244705,0.266833
2,HOBBIES_1_003_CA_1_validation,0.469926,0.439893,0.463241,0.464918,0.591388,0.710891,0.710500,0.458180,0.448215,...,0.565183,0.703196,0.769872,0.492762,0.461499,0.501418,0.526970,0.667997,0.787701,0.751642
3,HOBBIES_1_004_CA_1_validation,1.719050,1.404574,1.414955,1.523040,2.005390,3.067617,3.157289,1.794654,1.499168,...,1.945723,2.852580,3.570884,1.752903,1.418549,1.374881,1.396749,1.881594,3.037569,3.523097
4,HOBBIES_1_005_CA_1_validation,1.013351,0.874918,0.864337,0.883864,1.022494,1.379901,1.587213,0.958787,0.992209,...,1.107229,1.517535,1.622433,1.016027,0.868308,0.872556,0.863067,1.119161,1.514042,1.573210


In [10]:
# Reading competition sample submission and
# merging our predictions
# As we have predictions only for "_validation" data
# we need to do fillna() for "_evaluation" items
submission = pd.read_csv('sample_submission.csv')[['id']]
submission = submission.merge(all_preds, on=['id'], how='left').fillna(0)
submission.to_csv('xgb_bystore_clusterfe6_v5.csv', index=False)